In [ ]:
# --- Importar bibliotecas ---
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import widgets, VBox, HBox
from IPython.display import display

# --- Carregar dataset ---
df = pd.read_csv("C:/Users/SAMSUNG/Desktop/projeto1_10/SuperMarket Analysis.csv")

# --- Preparar colunas de data/hora ---
df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M', errors='coerce')
df['Hour'] = df['Time'].dt.hour
df['Week'] = df['Date'].dt.isocalendar().week
df['Month'] = df['Date'].dt.month

# --- Função do dashboard ---
def update_dashboard(city, product, payment):
    filtered = df.copy()
    if city != 'All':
        filtered = filtered[filtered['City'] == city]
    if product != 'All':
        filtered = filtered[filtered['Product line'] == product]
    if payment != 'All':
        filtered = filtered[filtered['Payment'] == payment]

    # --- KPIs ---
    total_sales = filtered['Sales'].sum()
    avg_ticket = filtered['Sales'].mean()
    max_sale = filtered['Sales'].max()
    num_transactions = filtered.shape[0]

    kpi_fig = go.FigureWidget()
    kpi_colors = ["#1f77b4", "#ff7f0e", "#2ca02c", "#d62728"]

    kpi_fig.add_trace(go.Indicator(
        mode="number+delta",
        value=total_sales,
        delta={'reference': df['Sales'].sum() - total_sales, 'relative': True, 'valueformat':'.0%'},
        title={"text": "💰 Receita Total"},
        number={'prefix': '$', 'font': {'size': 24, 'color': kpi_colors[0]}},
        domain={'x':[0,0.24],'y':[0,1]}
    ))
    kpi_fig.add_trace(go.Indicator(
        mode="number+delta",
        value=avg_ticket,
        delta={'reference': df['Sales'].mean(), 'relative': True, 'valueformat':'.0%'},
        title={"text": "🎟 Ticket Médio"},
        number={'prefix': '$', 'font': {'size': 24, 'color': kpi_colors[1]}},
        domain={'x':[0.26,0.5],'y':[0,1]}
    ))
    kpi_fig.add_trace(go.Indicator(
        mode="number",
        value=max_sale,
        title={"text": "🏆 Maior Venda"},
        number={'prefix': '$', 'font': {'size': 24, 'color': kpi_colors[2]}},
        domain={'x':[0.52,0.76],'y':[0,1]}
    ))
    kpi_fig.add_trace(go.Indicator(
        mode="number",
        value=num_transactions,
        title={"text": "🛒 Nº de Transações"},
        number={'font': {'size': 24, 'color': kpi_colors[3]}},
        domain={'x':[0.78,1],'y':[0,1]}
    ))
    kpi_fig.update_layout(height=160, margin=dict(t=20,b=20))

    # --- Gráficos como FigureWidget ---
    sales_time_fig = go.FigureWidget(px.line(
        filtered.groupby('Date')['Sales'].sum().reset_index(),
        x='Date', y='Sales',
        title='📈 Evolução das Vendas ao Longo do Tempo',
        markers=True
    ))
    sales_time_fig.update_traces(line_color='#1f77b4')

    hourly_fig = go.FigureWidget(px.bar(
        filtered.groupby('Hour')['Sales'].sum().reset_index(),
        x='Hour', y='Sales',
        title='⏰ Vendas por Hora do Dia',
        color='Sales', color_continuous_scale='Cividis'
    ))

    top_products_fig = go.FigureWidget(px.bar(
        filtered.groupby('Product line')['Sales'].sum().sort_values(ascending=False).head(5).reset_index(),
        x='Product line', y='Sales',
        title='🏅 Top 5 Produtos por Receita',
        text='Sales', color='Sales', color_continuous_scale='Viridis'
    ))
    top_products_fig.update_traces(texttemplate='%{text:.2f}', textposition='outside')

    heatmap_fig = go.FigureWidget(px.density_heatmap(
        filtered.groupby(['City','Product line'])['Sales'].sum().reset_index(),
        x='City', y='Product line', z='Sales',
        color_continuous_scale='Viridis',
        title='🌐 Heatmap Receita Cidade x Produto'
    ))

    # --- Layout do dashboard ---
    display(VBox([
        kpi_fig,
        HBox([sales_time_fig, hourly_fig]),
        HBox([top_products_fig, heatmap_fig])
    ]))

# --- Widgets interativos ---
city_widget = widgets.Dropdown(
    options=['All'] + df['City'].unique().tolist(),
    value='All',
    description='Cidade:'
)
product_widget = widgets.Dropdown(
    options=['All'] + df['Product line'].unique().tolist(),
    value='All',
    description='Produto:'
)
payment_widget = widgets.Dropdown(
    options=['All'] + df['Payment'].unique().tolist(),
    value='All',
    description='Pagamento:'
)

ui = HBox([city_widget, product_widget, payment_widget])
out = widgets.interactive_output(
    update_dashboard,
    {'city': city_widget, 'product': product_widget, 'payment': payment_widget}
)

display(ui, out)

#lotti <!=>


Output()